Import packages

In [ ]:
%matplotlib inline
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from IPython.display import clear_output  # this has to be removed if used on a single script
import random
from preprocess import preprocess_seismo, preprocess_coord
from scipy.ndimage.interpolation import shift
import GPy
import time
from sklearn.decomposition import PCA

# PCA compression

Define auxiliary functions

In [ ]:
def next_batch(x, y, mb_size):
    idx = np.arange(len(x), dtype = np.int64)
    np.random.shuffle(idx)
    idx = idx[:mb_size]
    return x[idx], y[idx], idx

def plot(x, index=0):
    fig = plt.figure(figsize=(10, 4))
    plt.plot(x, label='Reconstructed')
    plt.plot(X_data[index], label='Real')
    plt.legend()

def plot_test(x, index=0):
    fig = plt.figure(figsize=(10, 4))
    plt.plot(x, label='Reconstructed')
    plt.plot(X_data_test[index], label='Real')
    plt.legend()

def calculate_R2(original, prediction, label, store, component=None):
    AM = original.mean()
    BM = prediction.mean()
    c_vect = (original-AM)*(prediction-BM)
    d_vect = (original-AM)**2
    e_vect = (prediction-BM)**2
    r_out = np.sum(c_vect)/float(np.sqrt(np.sum(d_vect)*np.sum(e_vect)))
    if component == None:
        print(label+str(r_out))
    else:
        print(str(component)+label+str(r_out))
    store.append(r_out)

Define number of time components in seismograms, number of coordinates, train/test split and load data

In [ ]:
X_dim = 501 # size of the seismograms
y_dim = 4
# load data
split = 2000
test_valid = 1000
X_data_ = np.load('./seismograms_4000seismo_ISO.npy')[:, :X_dim]
y_data_ = np.load('./coordinates_4000seismo_ISO.npy')

In [ ]:
#start_time = time.time()

In [ ]:
# preprocess coordinates
y_data_preprocessed, meancoords, stdcoords = preprocess_coord(y_data_, split=split, test_valid=test_valid, sort=False, std=True)
y_data = y_data_preprocessed[:split]
y_data_valid =  y_data_preprocessed[split:split+test_valid]
y_data_test =  y_data_preprocessed[split+test_valid:]

In [ ]:
# preprocess seismograms
X_data_preprocessed = preprocess_seismo(X_data_, split, log=False, std=False, rescale=True, rescale_onlyamp=False)
X_data = X_data_preprocessed[:split]
X_data_valid =  X_data_preprocessed[split:split+test_valid]
X_data_test =  X_data_preprocessed[split+test_valid:]

Standardise data before PCA

In [ ]:
# we standardise data to apply PCA
meanseismo = np.mean(X_data, axis=0)
stdseismo = np.std(X_data, axis=0)
X_data = (X_data - meanseismo)/stdseismo
X_data_valid = (X_data_valid - meanseismo)/stdseismo
X_data_test = (X_data_test - meanseismo)/stdseismo

selected_comp = 20

pca = PCA(n_components=selected_comp)
pca.fit(X_data)
basis = pca.components_
dominantseismo_train = pca.transform(X_data)
dominantseismo_valid = pca.transform(X_data_valid)
#dominantseismo_test = pca.transform(X_data_test)
        
seismo = np.matmul(dominantseismo_train, basis)
seismo_valid = np.matmul(dominantseismo_valid, basis)
#seismo_test = dominantseismo_test @ basis

# plot training - don't think this is needed here
#fig, ax = plt.subplots(5, figsize=(15,30))
#for i in range(5):
#        ax[i].plot(X_data[100*i], color='blue', label='Original')
#        ax[i].plot(seismo[100*i], color='red', label='PCA')
#        ax[i].set_xlabel('Time components', fontsize='x-large')
#        ax[i].set_ylabel('Amplitude', fontsize='x-large')
#        ax[i].set_title('Seismogram n. {:}'.format(100*i+1), fontsize='x-large')
#        ax[i].legend(fontsize='large')
#plt.subplots_adjust(hspace=1)
#fig.savefig('IMAGES/PCA_rescale_{}dims_training.pdf'.format(dimension))
#plt.show()
#plt.close()

# R2
#AM = X_data.mean()
#BM = seismo.mean()
#c_vect = (X_data-AM)*(seismo-BM)
#d_vect = (X_data-AM)**2
#e_vect = (seismo-BM)**2
#r_out = np.sum(c_vect)/float(np.sqrt(np.sum(d_vect)*np.sum(e_vect)))
#np.savetxt('R2/r2_PCA_rescale_{}dims_training.txt'.format(dimension), r_out[None])
#print(r_out)

# GP training

In [ ]:
fitted_GP = np.zeros((test_valid, selected_comp))
# fit a GP to the components
for component in range(selected_comp):
    ker = GPy.kern.Matern32(4,ARD=True)
    m = GPy.models.GPRegression(y_data, dominantseismo_train[:, component].reshape(-1, 1), ker)
    m.optimize(messages=True,max_f_eval = 1000)

    y_pred_train = m.predict(y_data)[0]
    y_pred_train = y_pred_train[:, 0]
    calculate_R2(dominantseismo_train[:, component], y_pred_train, ' component R2 training: ', [], component)

    y_pred_valid = m.predict(y_data_valid)[0]
    y_pred_valid = y_pred_valid[:, 0]
    calculate_R2(dominantseismo_valid[:, component], y_pred_valid, ' component R2 validation: ', [], component)

    y_pred_test = m.predict(y_data_test)[0]
    fitted_GP[:, component] = y_pred_test[:, 0]

    np.save(f'./saved_models_iso_PCAplusGP/GPmodel_{component}.npy', m.param_array)
    #plt.plot(shift_index_test, color='blue')
    #plt.show()
    #plt.plot(y_pred_test_2, color='red')
    #plt.show()

# GP training for Amplitude and Time shifts

In [ ]:
amplitude_rescale = np.loadtxt('./amplitude_rescale_NOTsorted.txt')
shift_index = np.loadtxt('./shift_index_NOTsorted.txt')

In [ ]:
amplitude_rescale_train = amplitude_rescale[:split].reshape(X_data.shape[0], 1)
amplitude_rescale_valid = amplitude_rescale[split:split+test_valid]
amplitude_rescale_test = amplitude_rescale[split+test_valid:]

shift_index_train = shift_index[:split].reshape(X_data.shape[0], 1)
shift_index_valid = shift_index[split:split+test_valid]
shift_index_test = shift_index[split+test_valid:]

In [ ]:
# fit GP to data rescaling
# amplitude
kern = GPy.kern.Matern32(4,ARD=True)
n = GPy.models.GPRegression(y_data, amplitude_rescale_train, kern)
n.optimize(messages=True, max_f_eval = 1000)

y_pred_train = n.predict(y_data)[0]
y_pred_train = y_pred_train[:, 0]
calculate_R2(amplitude_rescale_train.flatten(), y_pred_train, 'Amplitude R2 train: ', [])

y_pred_valid = n.predict(y_data_valid)[0]
y_pred_valid = y_pred_valid[:, 0]
calculate_R2(amplitude_rescale_valid, y_pred_valid, 'Amplitude R2 validation: ', [])

# this is to be used later
y_pred_test = n.predict(y_data_test)[0]
y_pred_test = y_pred_test[:, 0]

#plt.plot(amplitude_rescale_test, color='blue')
#plt.show()
#plt.plot(y_pred_test, color='red')
#plt.show()

In [ ]:
# time shift
ker = GPy.kern.Matern32(4,ARD=True)
m = GPy.models.GPRegression(y_data,shift_index_train,ker)
m.optimize(messages=True,max_f_eval = 1000)

y_pred_train_2 = m.predict(y_data)[0]
y_pred_train_2 = y_pred_train_2[:, 0]
calculate_R2(shift_index_train.flatten(), y_pred_train_2, 'Time shift R2 training: ', [])

y_pred_valid_2 = m.predict(y_data_valid)[0]
y_pred_valid_2 = y_pred_valid_2[:, 0]
calculate_R2(shift_index_valid, y_pred_valid_2, 'Time shift R2 validation: ', [])

y_pred_test_2 = m.predict(y_data_test)[0]
y_pred_test_2 = y_pred_test_2[:, 0]

#plt.plot(shift_index_test, color='blue')
#plt.show()
#plt.plot(y_pred_test_2, color='red')
#plt.show()

In [ ]:
# obtain reconstructed seismo for the test set
prediction_testing = fitted_GP@basis

prediction_testing = prediction_testing*stdseismo+meanseismo

prediction_testing = np.multiply(prediction_testing, np.repeat(1/y_pred_test, X_dim).reshape(-1, X_dim))
for index_seism in range(test_valid):
    prediction_testing[index_seism] = shift(prediction_testing[index_seism], -y_pred_test_2[index_seism], cval=0.)

# retrieve the unprocessed data back
X_data_preprocessed = preprocess_seismo(X_data_, split, log=False, std=False, rescale=False, rescale_onlyamp=False)
X_data_test = X_data_preprocessed[split+test_valid:]

calculate_R2(X_data_test, prediction_testing, 'Final R2 testing: ', [])

#for i in range(5):
#    plt.plot(X_data_test[i], color='blue')
#    plt.plot(prediction_testing[i], color='red')
#    plt.show()

In [ ]:
# total time, to be quoted in the paper
#print("--- %s seconds ---" % (time.time() - start_time))

# Inference

In [ ]:
fitted_GP = []
# fit a GP to the components
for component in range(selected_comp):
    ker = GPy.kern.Matern32(4,ARD=True)
    m = GPy.models.GPRegression(y_data, dominantseismo_train[:, component].reshape(-1, 1), ker)
    m.update_model(False) # do not call the underlying expensive algebra on load
    m.initialize_parameter() # Initialize the parameters (connect the parameters up)
    m[:] = np.load('./saved_models_iso_PCAplusGP/GPmodel_{}.npy'.format(component)) # Load the parameters
    m.update_model(True) # do not call the underlying expensive algebra on load
    fitted_GP.append(m)

In [ ]:
ker = GPy.kern.Matern32(4,ARD=True) 
m_load = GPy.models.GPRegression(y_data, amplitude_rescale_train, ker, initialize=False)
m_load.update_model(False) # do not call the underlying expensive algebra on load
m_load.initialize_parameter() # Initialize the parameters (connect the parameters up)
m_load[:] = np.load('./saved_models_iso_PCAplusGP/GPmodel_amplitude.npy') # Load the parameters
m_load.update_model(True) # Call the algebra only once

kern = GPy.kern.Matern32(4,ARD=True) 
n_load = GPy.models.GPRegression(y_data, shift_index_train, kern, initialize=False)
n_load.update_model(False) # do not call the underlying expensive algebra on load
n_load.initialize_parameter() # Initialize the parameters (connect the parameters up)
n_load[:] = np.load('./saved_models_iso_PCAplusGP/GPmodel_time.npy') # Load the parameters
n_load.update_model(True) # Call the algebra only once

In [ ]:
start_time_inference = time.time()

coordinate = np.array([31,25,158])

shifted = coordinate - np.array([41,41,244])
distances = np.linalg.norm(shifted)
new_coords = np.zeros(4)
new_coords[:3] = shifted
new_coords[-1] = distances
new_coords = (new_coords - meancoords)/stdcoords
new_coords = new_coords.reshape((1,y_dim))

predPCA = np.zeros(selected_comp)
for i in range(selected_comp):
    predPCA_tmp = fitted_GP[i].predict(new_coords)[0]
    predPCA[i] = predPCA_tmp[:, 0]
prediction_testing = np.matmul(predPCA, basis)
prediction_testing = prediction_testing*stdseismo+meanseismo
y_pred_test = m_load.predict(new_coords)[0]
y_pred_test = y_pred_test[:, 0]
y_pred_test_2 = n_load.predict(new_coords)[0]
y_pred_test_2 = y_pred_test_2[:, 0]
prediction_testing = np.multiply(prediction_testing, np.repeat(1/y_pred_test, X_dim).reshape(-1, X_dim))
prediction_testing = shift(prediction_testing[0], -y_pred_test_2, cval=0.)

timeinf = time.time() - start_time_inference
print("timeinf", timeinf)
np.save("PCA_plus_GP_inftime.npy", timeinf)